In [119]:
# Importing libraries
import os
import music21 as m21
import json
import tensorflow.keras as keras
import numpy as np

In [120]:
# Loadind the data from application yaml
import yaml

with open("../config/application.yaml", "r") as file:
    config = yaml.safe_load(file)
    
dataset_path = config['dataset_path']
acceptable_durations = config['acceptable_durations']
save_path = config['save_path']
sequence_length = config['sequence_length']
mapping_file_path = config['mapping_file_path']

In [121]:
# defining a funtion to load all the files of the data set
songs = []

def load_songs_kern_dataset(dataset_path):
    for path, subdirs, files in os.walk(dataset_path):
        
        # we need to filter out the ".krn" files from the dataset
        for file in files:
            if file.endswith('.krn'):
                song = m21.converter.parse(os.path.join(path, file))
                songs.append(song)
    return songs

In [122]:
# Function to transpose a song to another scale
def transpose(song): 
    # get key from the song
    parts = song.getElementsByClass(m21.stream.Part)
    measure_part0 = parts[0].getElementsByClass(m21.stream.Measure)
    key = measure_part0[0][4]
    
    # if key not present then estimate the key using music21
    if not isinstance(key, m21.key.Key):
        key = song.analyze('key')
        
    # get the interval for transposition (example: BMaj to CMaj)
    if key.mode == 'major':
        interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch('C'))
    elif key.mode == 'minor':
        interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch('A'))
        
    # transpose the song
    transposed_song = song.transpose(interval)
    
    return transposed_song  

Here we will encode the song into code-understandable form. We will encode the pitch by numbers and duration by '_'. The encode_song function will convert the song to its encoded form. 

For example, if pitch is 60 and duration is 1.0 then this note will be encoded as:
[60, "\_", "\_", "\_"]

In [123]:
# Function to encode pitch and duration of song to machine-readable format
def encode_song(song, time_step=0.25):
    encoded_song = []
    for event in song.flat.notesAndRests:
        if isinstance(event, m21.note.Note):
            symbol = event.pitch.midi
        elif isinstance(event, m21.note.Rest):
            symbol = 'r'
            
        # convert the notes and rests into time series notation
        steps = int(event.duration.quarterLength/time_step)
        for step in range(steps):
            if step == 0:
                encoded_song.append(symbol)
            else:
                encoded_song.append('_')
                
    # Calculate the duration of the song
    encoded_song = " ".join(map(str, encoded_song))
    
    return encoded_song

In [124]:

# function to preproces the songs dataset and prepare the data for our model

def preprocess_songs(dataset_path):
    # Load the songs from the dataset
    print("Loading songs from dataset...")
    songs = load_songs_kern_dataset(dataset_path)
    print(f"Loaded {len(songs)} songs from the dataset.")
    
    # Filter songs based on acceptable durations
    print("Filtering songs based on acceptable durations...")
    for i, song in enumerate(songs):
        if not has_acceptable_duration(song, acceptable_durations):
            continue
    
        # Transpose song to C major or A minor
        song = transpose(song)
        
        # Encode songs with music time series representation
        encoded_song = encode_song(song)   
        
        # Save the encoded song to a file in save path
        saved_path = os.path.join(save_path,  f"song_{i}.txt")
        with open(saved_path, 'w') as fp:
            fp.write(encoded_song)

In [125]:
# Function to check whether a song has acceptable duration
def has_acceptable_duration(song, acceptable_durations):
    for note in song.flat.notesAndRests:
        if note.duration.quarterLength not in acceptable_durations:
            return False
    return True

In [126]:
def load(file_path):
    with open(file_path, 'r') as fp:
        song = fp.read()
    return song

In [127]:
# Creating a single file that will contain all the songs
def create_single_file_dataset(dataset_path, sequence_length):
    new_song_delimiter = "/ " * sequence_length
    songs = ""
    
    # Load encoded songs and add delimiters
    for path, _, files in os.walk(dataset_path):
        for file in files:
            file_path = os.path.join(path, file)
            song = load(file_path)
            songs = songs + song + " " + new_song_delimiter
            songs = songs[:-1]     # remove the last delimiter to avoid trailing spaces
            
    # save the string in the single file
    dataset_path1 = os.path.join(dataset_path,  f"songs.txt")
    with open(dataset_path1, "w") as fp:
        fp.write(songs)
        
    return songs

In [128]:
# Creating a lookup table for all the symbols used in the final dataset
def create_mapping(songs, mapping_file_path):
    mappings = {}
     
    # Identify the vocabluary
    songs = songs.split()
    vocabulary = list(set(songs))
     
    # Create mappings
    for i, symbol in enumerate(vocabulary):
        mappings[symbol] = i
         
    # save the vocalbuary to a file
    with open(mapping_file_path, 'w') as fp:
        json.dump(mappings, fp, indent=4)

In [129]:
def convert_songs_to_int(songs):
    int_songs = []
    
    # Load mappings
    with open(mapping_file_path, 'r') as fp:
        mappings = json.load(fp)
        
    # Cast song string to a list
    songs = songs.split()
    
    # Map song to the int
    for symbol in songs:
        int_songs.append(mappings[symbol])
    
    return int_songs

In [ ]:
def generating_training_sequences(sequence_length):
    # Load songs and map them to integers
    single_file_dataset = save_path + "/songs.txt"
    songs = load(single_file_dataset)
    int_songs = convert_songs_to_int(songs)
    
    # Generate training sequences
    inputs = []
    targets = []
    num_sequences = len(int_songs) - sequence_length
    for i in range(num_sequences):
        inputs = np.append(inputs, int_songs[i:i + sequence_length])
        targets = np.append(targets, int_songs[i + sequence_length])
        inputs = inputs[:50000]  # Limit the number of inputs to 50000
        targets = targets[:50000]  # Limit the number of targets to 50000
        
        # One hot encoding the sequence
        vocabulary_size = len(set(int_songs))
        inputs = keras.utils.to_categorical(inputs, num_classes=vocabulary_size)
        targets = np.array(targets)
        
    return inputs, targets

In [131]:
def main():
    preprocess_songs(dataset_path)
    songs = create_single_file_dataset(save_path, sequence_length)
    create_mapping(songs, mapping_file_path)
    inputs, targets = generating_training_sequences(sequence_length)

In [132]:
if __name__ == "__main__":
    main()

Loading songs from dataset...
Loaded 12 songs from the dataset.
Filtering songs based on acceptable durations...


MemoryError: Unable to allocate 28.3 GiB for an array with shape (345538752, 22) and data type float32